In [1]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import netCDF4
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import colormaps as cmaps
import calc_date as cd
import save_fig as SF
import createFolder as cF



In [2]:
nc_dir = '../../Retrieval/MRR/MRR_data'
fig_dir = '../../retrieval_fig/MRR_ref'
cF.createFolder(fig_dir)
form = 'png'

sfig = 1

year = '2016'
mon = '12'
t = ['20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
#t = ['25']

fs = 22
#file_out = 'VMRR_data_%s%s%s.txt' %(year,mon,day)

In [3]:
date_blue = np.array([1,74,159])/255.           # for the date

In [4]:
for day in t:
    fn = netCDF4.Dataset('%s/MRR_HKL_%s%s%s_maahn_r01.nc' %(nc_dir,year,mon,day) ,'r')

    fn

    calday, calmon = cd.get_dayname(year,mon,day)
    
# Read in varaibel 'time' (time=1440)
    time = fn.variables['time']


    netCDF4.num2date(time[:],time.units)

# Read in variable 'Ze' (time = 1440,range = 31)
    Ze = fn.variables['Ze']

# fill value, where Ze = -9999, fill_value = -50.
    mask = np.ma.getmaskarray(Ze[:,:])
    Ze = np.ma.array(Ze[:,:], mask = mask, fill_value = -50.)
    Ze = Ze.filled()

    Z = 10**(Ze / 10.)

# no. of layers
    nlay = 14

# bottom of the layer(start of valid MRR measurements at 300) 
    bot = []
# top of the layer (layer thickness will be 200)
    top = []

    for k in range(0, nlay):
        bot.append(300.+ k*200)
        top.append(bot[k]+200)

# Read in variable 'height' (time =1440,range = 31)
    height = fn.variables['height']


    avgref = []
    h_mid = []
    tid = []
    for i in range(0, height.shape[0]):
        for k in range(0, nlay):
            idx = np.where(np.logical_and(height[i,:] >= bot[k], height[i,:] < top[k]))
        # mean reflection in layer
            res = np.mean(Z[i,idx])
        # average reflection in layer transformed in dBz
            avgref.append(10*math.log10(res))
            h_mid.append((bot[k]+top[k])/2)
            tid.append(time[i]-time[0])


    avgref = np.split(np.asarray(avgref),height.shape[0])
    h_mid = np.split(np.asarray(h_mid),height.shape[0])

    tid = np.split(np.asarray(tid),height.shape[0])

    levels = np.arange(-10,30,0.2)
    fig = plt.figure(figsize=(20.,12))
    ax0 = fig.add_subplot(111)
    CS = ax0.contourf(np.asarray(tid), np.asarray(h_mid) , np.asarray(avgref), 
                  levels, cmap='jet')
# add colorbar
    cbaxes = fig.add_axes([0.14, 0.03, .75, .02] )   #[left, bottom, width, height] 
    cbar = plt.colorbar(CS, orientation = 'horizontal', cax=cbaxes)
    cbar.ax.set_xlabel('MRR reflectivity [dBz]', fontsize = 22)
    cbar.ax.tick_params(labelsize = 20)

# labels
    times = [0, 3, 6, 9,12, 15, 18, 21, 24]
    ax0.set_xticks(np.arange(0,60*60*25,3*60*60))
    ax0.set_xticklabels(times, fontsize = 20)
    ax0.set_xlabel('time [hours]', fontsize = 22)

    ax0.set_ylabel('height [km]', fontsize = 22)
    ax0.set_ylim(0,3.5)
    ax0.set_yticks(np.arange(0,3500.,500.))
    yl = [0., '' , 1.0, '' , 2., '' , 3.]
    ax0.set_yticklabels(yl, fontsize = 20)
    
    
# textbox
    ax0.text(0.02,0.98, '%s, %s %s %s' %(calday, day, calmon, year), verticalalignment = 'top',  
         horizontalalignment='left',
             transform = ax0.transAxes,
            color =date_blue, fontsize=fs,
           bbox={'facecolor':'white','alpha':1., 'pad':10})
    if sfig == 1:
        fig_name = 'MRR_%s%s%s.%s' %(year,mon,day,form)
        SF.save_figure_landscape(fig_dir, fig_name, form)
    else:
        plt.show()
    plt.close()
    fn.close()

/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in less
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in greater_equal
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [5]:
# write netCDF file including tid, h_mid, avgref